# 5.1 CNN으로 패션 아이템 구분하기
Convolutional Neural Network (CNN) 을 이용하여 패션아이템 구분 성능을 높여보겠습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
torch.manual_seed(42)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:05, 1866240.63it/s]                             


Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 114062.14it/s]           
0it [00:00, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1202736.42it/s]                             
0it [00:00, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 41937.82it/s]            

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## 뉴럴넷으로 Fashion MNIST 학습하기

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## 하이퍼파라미터 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## 학습하기

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

In [8]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # sum up batch loss
            test_loss += F.cross_entropy(output, target,
                                         size_average=False).item()

            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 학습이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310367
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.561445
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.777584
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.468698
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.666155


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[1] Test Loss: 0.2048, Accuracy: 93.96%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.671450
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.414416
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.520516
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.131479
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.326400
[2] Test Loss: 0.1312, Accuracy: 96.02%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.296757
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.433390
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.516062
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.276350
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.171822
[3] Test Loss: 0.1036, Accuracy: 96.85%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.263187
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.143500
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.279130
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.571077
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.229072
[4] Test Loss: 0.0907, Accuracy: 97.33%
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.226478
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.330274
Train Epoc

Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.060862
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.071246
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.053050
[30] Test Loss: 0.0382, Accuracy: 98.89%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.178070
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.030166
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.123735
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.090055
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.209241
[31] Test Loss: 0.0366, Accuracy: 98.88%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.079730
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.041189
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.091911
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.058561
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.038800
[32] Test Loss: 0.0341, Accuracy: 99.00%
Train Epoch: 33 [0/60000 (0%)]	Loss: 0.079959
Train Epoch: 33 [12800/60000 (21%)]	Loss: 0.095628
Train Epoch: 33 [25600/60000 (43%)]	Loss: 0.071483
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.076589
Train Epoch: 33 [51200/60